In [50]:
import geopandas as gpd
gpd.options.use_pygeos = True

import shapely
import matplotlib.pyplot as plt

from secrets.paths import alle_co_path

from sqlalchemy import create_engine
from secrets.con_str import con_str

engine = create_engine(con_str)

In [2]:
parcels_shp_path = f'{alle_co_path}/AlleghenyCounty_Parcels202202/AlleghenyCounty_Parcels202202.shp'
parcels_gdf = gpd.read_file(parcels_shp_path)

In [3]:
parcels_gdf

,PIN,MAPBLOCKLO,MUNICODE,CALCACREAG,NOTES,PSEUDONO,COMMENTS,GlobalID,Shape_Leng,Shape_Area,geometry
0,0102P00193000000,102-P-193,941,0.07,None,None,None,{96966DF2-4060-43D0-9516-FD5C97479B43},306.498674,3251.155592,"POLYGON ((1315443.201 395608.825, 1315417.701 ..."
1,0124F00169000000,124-F-169,112,0.02,None,None,None,{76720B1A-9E6C-43B5-9270-8F0D63312477},149.703009,852.003350,"POLYGON ((1367397.828 420679.928, 1367385.145 ..."
2,0006K00358000000,6-K-358,119,0.04,None,None,None,{41F542B5-C253-4E7F-B816-A4B3146471E7},235.746576,1791.194106,"POLYGON ((1335443.757 409727.695, 1335436.131 ..."
3,0858M00274000000,858-M-274,879,0.25,None,None,None,{33A3EE8B-38B1-4576-B747-AAB14334FEF4},430.870823,10730.808711,"POLYGON ((1411227.875 401626.749, 1411331.750 ..."
4,0407J00004000000,407-J-4,946,0.83,None,9946X01562000000,None,{81861533-11EF-44B6-8596-7AC99E8F7CBB},1260.436214,36243.436135,"POLYGON ((1292132.946 397263.813, 1292079.351 ..."
...,...,...,...,...,...,...,...,...,...,...,...
583672,0112K00391000000,112-K-391,949,0.07,None,None,None,{2B3685B0-3B0D-4EFA-B57B-A7E17D69E9FF},300.713223,3134.439937,"POLYGON ((1321883.963 428663.903, 1321861.819 ..."
583673,0471F00124000000,471-F-124,873,0.00,None,None,None,{3ACD0530-DB6A-447B-8FEF-0AF1BD6FB42E},370.634445,7316.041368,"POLYGON ((1352616.250 371627.566, 1352528.492 ..."
583674,0176E00201000000,176-E-201,114,0.10,<Null>,None,None,{42FA29D2-1CE4-4186-9BD9-DC6E91FE8CCA},313.910366,4205.542206,"POLYGON ((1371361.875 410084.563, 1371367.500 ..."
583675,0464A00259000000,464-A-259,411,0.07,None,None,None,{82A46022-6315-4854-B41D-A405987ED331},313.161241,3171.895227,"POLYGON ((1383968.539 371727.327, 1383953.780 ..."


In [59]:
parcels_gdf['geom'].geom_type.unique()

array(['Polygon', 'MultiPolygon'], dtype=object)

In [16]:
invalid_geom = parcels_gdf.loc[~parcels_gdf.is_valid, 'geometry']

In [20]:
from shapely.validation import explain_validity
from shapely.validation import make_valid

invalid_geom.apply(explain_validity)

7609      Ring Self-intersection[1362793.01579675 404184...
12467     Ring Self-intersection[1352330.98377703 480144...
24100     Ring Self-intersection[1352330.98377703 480144...
29872     Ring Self-intersection[1303051.59176968 379498...
39720     Ring Self-intersection[1377495.87575281 415411...
65087     Too few points in geometry component[1416977.2...
66718     Ring Self-intersection[1421552.93831384 475506...
74774     Ring Self-intersection[1347570.31429981 458494...
107269    Ring Self-intersection[1350561.53490052 492631...
109381    Ring Self-intersection[1352330.98377703 480144...
109710    Ring Self-intersection[1335665.3386112 494782....
118149    Too few points in geometry component[1381304.7...
126641    Too few points in geometry component[1314264.9...
131003    Self-intersection[1387266.49618826 337192.7347...
155460    Self-intersection[1334039.04053143 485371.5421...
182046    Self-intersection[1288447.74882541 456235.7223...
187070    Too few points in geometry com

In [22]:
made_valid_geom = invalid_geom.apply(make_valid)

In [40]:
made_valid_geom.geom_type.unique()

array(['Polygon', 'MultiPolygon', 'GeometryCollection'], dtype=object)

In [36]:
from shapely.ops import unary_union

made_valid_no_ls_geom = made_valid_geom.copy()

for i in range(len(made_valid_geom)):
  valid_geom = made_valid_geom.iloc[i]
  g_type = valid_geom.geom_type
  
  if g_type == "GeometryCollection":
    # p = gpd.GeoSeries(valid_geom)
    # p.plot()
    # plt.show()
    sub_geoms = []
    for sub_geom in valid_geom.geoms:
      sub_g_type = sub_geom.geom_type
      if sub_g_type in ["Polygon", "MultiPolygon"]:
        sub_geoms.append(sub_geom)
    made_valid_no_ls_geom.iloc[i] = unary_union(sub_geoms)
    # p = gpd.GeoSeries(unary_union(sub_geoms))
    # p.plot()
    # plt.show()

In [39]:
made_valid_no_ls_geom.geom_type.unique()

array(['Polygon', 'MultiPolygon'], dtype=object)

In [46]:
parcels_gdf.loc[made_valid_no_ls_geom.index, 'geometry'] = made_valid_no_ls_geom

In [48]:
parcels_gdf.is_valid.unique()

array([ True])

In [56]:
parcels_gdf.columns = [col.lower() for col in parcels_gdf.columns]
parcels_gdf.rename(columns={'geometry':'geom'}, inplace=True)
parcels_gdf.set_geometry(col='geom', inplace=True)
parcels_gdf.columns

Index(['pin', 'mapblocklo', 'municode', 'calcacreag', 'notes', 'pseudono',
       'comments', 'globalid', 'shape_leng', 'shape_area', 'geom'],
      dtype='object')

In [57]:
parcels_gdf.to_postgis(
  name='parcels_fixed',
  con=engine,
  if_exists='replace'
)